In [1]:
import pandas as pd
import numpy as np
import datetime
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline 
from random import sample
from functools import reduce
import re

In [2]:
path = '../data/'

### Slide `pickup_taxizone_id` column up by 1 within key (`hack_license`, `medallion`)

In [3]:
# path = '../data/'
# conn = sqlite3.connect(path + 'trip_data.db')

# q_license = """
# SELECT distinct(hack_license) as UNIQ_LIC
# FROM trip
# """

# unique_license = pd.read_sql(q_license, conn)

# LIC_SAMPLE_NUM = 10
# sampled_drivers = sample(unique_license.UNIQ_LIC.tolist(), LIC_SAMPLE_NUM)

# # find the pickup and dropoff time of next trip
# q_trip = """
# SELECT dropoff_datetime, dropoff_taxizone_id, 
#         lag(pickup_taxizone_id, -1, NULL) over (partition by hack_license, medallion order by pickup_datetime) as pickup_taxizone_id_next,
#         lag(pickup_datetime, -1, NULL) over (partition by hack_license order by pickup_datetime) as pickup_datetime_next
#     FROM trip
#     WHERE hack_license in {0}
# ORDER BY hack_license ASC, medallion ASC, pickup_datetime ASC, dropoff_datetime ASC
# """

# trip_df = pd.read_sql(q_trip.format(tuple(sampled_drivers)), conn)
# conn.close()

In [4]:
# path = '../data/'
# conn = sqlite3.connect(path + 'trip_data.db')

# # find the pickup and dropoff time of next trip
# q_trip = """
# SELECT dropoff_datetime, dropoff_taxizone_id, 
#         lag(pickup_taxizone_id, -1, NULL) over (partition by hack_license, medallion order by pickup_datetime) as pickup_taxizone_id_next,
#         lag(pickup_datetime, -1, NULL) over (partition by hack_license order by pickup_datetime) as pickup_datetime_next
#     FROM trip
# ORDER BY hack_license ASC, medallion ASC, pickup_datetime ASC, dropoff_datetime ASC
# LIMIT 10
# """

# trip_df = pd.read_sql(q_trip, conn)
# conn.close()

### DF processing

* Load cleaned trip df
* shift pickup time and taxi zone 1 row over
* keep only next pickup = current dropoff zone

In [5]:
cleaned_trip_df = pd.read_csv('../data/trip_cleaned.csv')



In [6]:
trip_df = cleaned_trip_df[['hack_license', 'pickup_taxizone_id', 'dropoff_taxizone_id',
                           'pickup_datetime', 'dropoff_datetime']].copy()

In [7]:
trip_df

,hack_license,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime,dropoff_datetime
0,2013000001,148.0,79.0,2013-06-24 18:55:00,2013-06-24 19:01:00
1,2013000001,164.0,143.0,2013-06-24 19:12:00,2013-06-24 19:30:00
2,2013000001,142.0,79.0,2013-06-24 19:45:00,2013-06-24 20:10:00
3,2013000001,234.0,141.0,2013-06-24 20:17:00,2013-06-24 20:28:00
4,2013000001,141.0,263.0,2013-06-24 20:30:00,2013-06-24 20:35:00
...,...,...,...,...,...
13885734,2013038800,45.0,79.0,2013-06-30 22:10:00,2013-06-30 22:18:00
13885735,2013038800,144.0,65.0,2013-06-30 22:29:00,2013-06-30 22:38:00
13885736,2013038800,144.0,13.0,2013-06-30 22:53:00,2013-06-30 23:00:00
13885737,2013038800,45.0,148.0,2013-06-30 23:08:00,2013-06-30 23:17:00


* Shift PU time and zone
* keep only rows whose DO zone == next PU zone

In [8]:
trip_df.sort_values(['hack_license', 'pickup_datetime'], inplace=True)
trip_df['pickup_datetime_next'] = trip_df.groupby(['hack_license'])['pickup_datetime'].shift(-1)
trip_df['pickup_taxizone_id_next'] = trip_df.groupby(['hack_license'])['pickup_taxizone_id'].shift(-1)

trip_df = trip_df.loc[trip_df['dropoff_taxizone_id'] == trip_df['pickup_taxizone_id_next']].copy()
trip_df.dropna(subset = ['pickup_taxizone_id_next'], inplace=True)

In [9]:
trip_df

,hack_license,pickup_taxizone_id,dropoff_taxizone_id,pickup_datetime,dropoff_datetime,pickup_datetime_next,pickup_taxizone_id_next
3,2013000001,234.0,141.0,2013-06-24 20:17:00,2013-06-24 20:28:00,2013-06-24 20:30:00,141.0
4,2013000001,141.0,263.0,2013-06-24 20:30:00,2013-06-24 20:35:00,2013-06-24 20:37:00,263.0
5,2013000001,263.0,239.0,2013-06-24 20:37:00,2013-06-24 20:49:00,2013-06-24 20:52:00,239.0
8,2013000001,162.0,233.0,2013-06-26 17:24:00,2013-06-26 17:35:00,2013-06-26 17:36:00,233.0
12,2013000001,233.0,114.0,2013-06-26 18:28:00,2013-06-26 18:50:00,2013-06-26 18:53:00,114.0
...,...,...,...,...,...,...,...
13885719,2013038799,224.0,224.0,2013-06-30 19:49:31,2013-06-30 19:55:13,2013-06-30 21:12:41,224.0
13885720,2013038799,224.0,224.0,2013-06-30 21:12:41,2013-06-30 21:30:09,2013-06-30 21:31:35,224.0
13885721,2013038799,224.0,224.0,2013-06-30 21:31:35,2013-06-30 21:47:59,2013-06-30 21:59:27,224.0
13885722,2013038799,224.0,224.0,2013-06-30 21:59:27,2013-06-30 22:21:20,2013-06-30 22:24:04,224.0


### Compute cruise time

In [10]:
trip_df['pickup_datetime_next'] = pd.to_datetime(trip_df['pickup_datetime_next'])
trip_df['dropoff_datetime'] = pd.to_datetime(trip_df['dropoff_datetime'])

trip_df['cruise_time'] = (trip_df['pickup_datetime_next'] - trip_df['dropoff_datetime'])/np.timedelta64(1, 'm')
trip_df.reset_index(inplace=True)


In [11]:
k = 5
print('Cruise Time less than 0 minutes: {}'.format(trip_df.loc[(trip_df['cruise_time'] < 0)].shape[0]))
print('Cruise Time greater than {} minutes: {}'.format(k*60, trip_df.loc[(trip_df['cruise_time'] >= k*60)].shape[0]))
print('These records are dropped')

Cruise Time less than 0 minutes: 525
Cruise Time greater than 300 minutes: 16065
These records are dropped


### Remove records with cruise time > 300 mins or <0 

In [12]:
trip_df = trip_df.loc[(trip_df['cruise_time'] <= k*60) &
                                        (trip_df['cruise_time'] >= 0)]

# trip_df = trip_df.astype({'cruise_time': 'int32'})

In [13]:
# trip_df.to_csv('../data/trip_df.csv', index=False)

## [aside] Coversion table 

In [29]:
def _generate_one_interval_conversion(interval):
    assert 24*60%interval==0, 'must provide even time interval'
    time_index_list = []
    time_interval_list = []
    time_index_current = 0
    interval_per_day = int(24*60/interval)
    for it in range(interval_per_day):
        minute = (it * interval) % 60 
        hour = (it * interval) // 60 
        time_interval_list.append(datetime.time(hour, minute))
        time_index_list.append(time_index_current)
        time_index_current += 1
    index_col_name = 'time_index_{}m'.format(int(interval))
    interval_index_table = pd.DataFrame({'interval': time_interval_list, 
                                         index_col_name: time_index_list}) 
    return interval_index_table

def generate_interval_conversion(interval_list, file_name=None):
  
    df_list = []
    for it in interval_list:
        df_list.append(_generate_one_interval_conversion(it))
    conv_table = reduce(lambda x, y: pd.merge(x, y, how = "outer", on = 'interval'), 
                        df_list)
    
    ##forward fill and change type to int
    conv_table.ffill(inplace=True)
    int_cols = conv_table.columns.drop('interval')
    for c in int_cols:
        conv_table[c] = conv_table[c].astype('int32')
    
    if file_name is not None: 
        conv_table.to_csv(file_name, index=False)
        print('table saved at ', file_name)
    return conv_table

In [30]:
interval = [1, 5, 10, 15, 30, 60]
file_name = path + 'interval_index_table_0.csv'
interval_index_table = generate_interval_conversion(interval, file_name)



table saved at  ../data/interval_index_table_0.csv


In [32]:
interval_index_table

,interval,time_index_1m,time_index_5m,time_index_10m,time_index_15m,time_index_30m,time_index_60m
0,00:00:00,0,0,0,0,0,0
1,00:01:00,1,0,0,0,0,0
2,00:02:00,2,0,0,0,0,0
3,00:03:00,3,0,0,0,0,0
4,00:04:00,4,0,0,0,0,0
...,...,...,...,...,...,...,...
1435,23:55:00,1435,287,143,95,47,23
1436,23:56:00,1436,287,143,95,47,23
1437,23:57:00,1437,287,143,95,47,23
1438,23:58:00,1438,287,143,95,47,23


In [35]:
"""
Give df with cruise time columm and an interval conversion table, compute
mean and median of cruise time for certain taxi zone and timestamp. The output
is saved as `cruise_time_{}m.csv`
"""
def compute_cruise_time(trip_df, interval_index_table, file_dir):
    
    ## extract time interval info from interval_index_table
    current_cruise_time = []
    index_cols = interval_index_table.columns.values[1:]
    time_index_list = [int(re.search(r'\d+', t).group(0)) for t in index_cols]
    
    for current_i in range(len(index_cols)):
        ## Extract current time interval and column name
        current_index_cols = index_cols[current_i]
        current_time_interval = int(time_index_list[current_i])

        ## round DO time to the same interval
        current_trip_df = trip_df.copy()
        round_by = '{}min'.format(current_time_interval)
        current_trip_df['dropoff_datetime_interval'] = current_trip_df['dropoff_datetime'].dt.round(round_by).dt.time

        ## convert DO interval to time index
        current_conversion = dict(zip(interval_index_table.interval, interval_index_table[current_index_cols]))
        current_trip_df['dropoff_datetime_index'] = [current_conversion[t] for t in current_trip_df['dropoff_datetime_interval']]

        ## Compute med and mean of cruise time
        current_cruise_time = current_trip_df.groupby(['dropoff_datetime_index', 
                                                       'dropoff_datetime_interval', 
                                                       'dropoff_taxizone_id'], as_index=False)['cruise_time'].agg(['mean', 'median', 'count']).reset_index()
        ##round stat to int
        current_cruise_time['mean'] = [round(t) for t in current_cruise_time['mean']]
        current_cruise_time['median'] = [round(t) for t in current_cruise_time['median']]
        
        ## cast the stat to int
        current_cruise_time = current_cruise_time.astype({'mean': 'int32', 'median': 'int32',
                                                          'dropoff_taxizone_id': 'int32',
                                                          'dropoff_datetime_index': 'int32'})

        ## rename columns
        current_cruise_time.rename(columns = {'dropoff_taxizone_id': 'taxizone_id',
                                          'mean': 'avg_cruise_time'.format(current_time_interval),
                                          'median': 'med_cruise_time'.format(current_time_interval)}, 
                                   inplace=True)
        
        ## Convert stat to interval
        current_cruise_time['avg_cruise_time_INT'] = [t//current_time_interval for t in current_cruise_time['avg_cruise_time']]
        current_cruise_time['med_cruise_time_INT'] = [t//current_time_interval for t in current_cruise_time['med_cruise_time']]


        ## export files
        if file_dir is not None:
            final_file_name = file_dir + 'cruise_time_{}m.csv'.format(current_time_interval)
            current_cruise_time.to_csv(final_file_name, index=False)
            print('file saved at ', final_file_name)
        
        current_cruise_time.append(current_cruise_time)
        
    return current_cruise_time

In [36]:
cruise_time_result = compute_cruise_time(trip_df, interval_index_table, path)

file saved at  ../data/cruise_time_1m.csv
file saved at  ../data/cruise_time_5m.csv
file saved at  ../data/cruise_time_10m.csv
file saved at  ../data/cruise_time_15m.csv
file saved at  ../data/cruise_time_30m.csv
file saved at  ../data/cruise_time_60m.csv


## ====